In [1]:
import re
import pdfplumber
import pandas as pd
from collections import namedtuple
import glob
import os

In [102]:
Line = namedtuple("Line","CID10 DESC JANEIRO FEVEREIRO MARÇO ABRIL MAIO JUNHO JULHO AGOSTO SETEMBRO OUTUBRO NOVEMBRO DEZEMBRO")
line_re =re.compile(r'(^\w{1}\d{2}[*†]*|[^\-\w]) ([\s\S]+(?<=[A-Za-z(),[\]\"áàâãéèêíïóôõöúçñÁÀÂÃÉÈÊÍÏÓÔÕÖÚÇÑ])) ([\s\S]+(?![0-9 -]))', flags=2)
pdffiles = []
pdffiles = [os.path.basename(x) for x in glob.glob('data\\*.pdf')]
for cNome in pdffiles:
    file="data\\"+cNome
    lines =[]
    total_check =0 
    with pdfplumber.open(file) as pdf:
        pages = pdf.pages
        for page in pdf.pages:
            text= page.extract_text(x_tolerance=5, y_tolerance=2)
            for line in text.split('\n'):
                comp =line_re.search(line)
                if comp:
                    CID, CID_DES, DADOS = comp.group(1), comp.group(2) , comp.group(3)
                    DADOS = DADOS.replace(".","")
                    DADOS = re.sub(r'(\d)\s(\d)', r'\1\2', DADOS)
                    items = DADOS.split()
                    if(len(items)==12):
                        lines.append(Line(CID,CID_DES,*items))
    dfCat =  pd.read_csv("data//CID-10-CAPITULOS.csv",delimiter=";",encoding='cp1252')
    dfCat.drop(columns='Unnamed: 5',axis=1,inplace=True)
    df = pd.DataFrame(lines)
    cols = df.iloc[:,2:len(df)].columns
    df.iloc[:,2:len(df)].replace("–",0,inplace=True)
    df[cols] = df[cols].apply(pd.to_numeric,errors='coerce')
    df = df.assign(key=1).merge(dfCat.assign(key=1), on='key')\
                 .drop(columns='key')\
                 .query("CID10.between(CATINIC,CATFIM)|CID10.str.contains('–')")\
                 .drop(columns=['CATINIC', 'CATFIM','DESCRICAO','DESCRABREV'])\
                 .reset_index(drop=True)
    df.drop_duplicates(subset=['CID10','JANEIRO','FEVEREIRO','MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO','DEZEMBRO'],keep='first',inplace=True)
    df = df.append(df.sum(numeric_only=True), ignore_index=True)
    cNome =cNome.replace(".pdf","")
    df.to_excel(cNome+".xlsx",sheet_name=cNome)
    df.to_csv(cNome+".csv")

NameError: name 'os' is not defined